In [23]:
import numpy as np
import imageio
from matplotlib import pyplot as plt

In [24]:
# TODOS:
# Is there a way to smooth out the levels? Averaging surrounding pixel values, like gaussian blur
# Is there a way to control the resolution of pixel detail independently of the input image resolution?
# ?include ability to input a depth_map strip
# ?output in color, random or like cmaps
# ?make a mild structure to the random dot strip, randomize 4/5 rows and every 5th has structure
# ?half width and double the depth_map repetition so the construction is not visible

In [25]:
# Helper function.
def image_read(filename='saturn.png'):
    """Use imageio to read depth map."""
    depth_map = imageio.imread(filename)
    if len(depth_map.shape) == 4: #RGBA
        depth_map = np.mean(depth_map[:,:,:3], axis=2)
    elif len(depth_map.shape) == 3: #RGB
        depth_map = np.mean(depth_map, axis=2)
    return depth_map

In [26]:
# Helper function for make_background().
def random_dot_background(m_rows, far_wi):
    """Generate strip of random dot values."""
    strip_width = far_wi
#     background = np.random.uniform(0, 1, size=(m_rows, strip_width))
    background = np.random.normal(0, .1, size=(m_rows, strip_width))
    background = np.cumsum(np.cumsum(background, axis=0), axis=1) % 1
    
#     # Color Variance (bigger means larger shifts).
#     cvar = .1
#     roff = 0
#     rowsoff = [0,]
#     for _ in range(m_rows - 1):
#         roff += np.random.uniform(-1, 1) * cvar
#         rowsoff.append(roff)
#     background = (background + np.array(rowsoff).reshape((m_rows,1))) % 1
    return background

In [27]:
# Helper function.
def make_background(stripe, cmap, m_rows, far_wi):
    """Create Background starter with m_rows rows of color."""
    if stripe is None:
        background = random_dot_background(m_rows, far_wi)
        return background
    strip = stripe[:,:,:3]
    dim = strip.shape
    j, k = dim[0], dim[1]
    # Width <= 140 is for good viewing.
#     far_wi = max((far_wi, k))
    repeat_rows = m_rows // j + 1
    repeat_cols = far_wi // k + 1
    # Grayscale must tile differently.
    if cmap is None:
        background = np.tile(strip, [repeat_rows, repeat_cols, 1])[:m_rows,:far_wi]
    else:
        strip = np.mean(strip, axis=2)
        background = np.tile(strip, [repeat_rows, repeat_cols])[:m_rows,:far_wi]
    return background

In [28]:
# Helper function.
#### ?not finished for (m_rows,n_cols,p) background
def gen_rows(stripe, backg, off_map, m_rows, wi, cmap):
    """Generate the image using depth map as pixel offset."""
    if stripe is not None and cmap is None:
        for i in range(wi):
            # Fancy Indexing: col = background[ rows, cols ] (i.e. specify wanted pixels).
            col = backg[np.arange(m_rows), i + off_map[:,i]].reshape((m_rows,1,3))
            backg = np.append(backg, col, axis=1)
            
    else:
        for i in range(wi):
            # Fancy Indexing: col = background[ rows, cols ].
            col = backg[np.arange(m_rows), i + off_map[:,i]].reshape((m_rows,1))
            backg = np.append(backg, col, axis=1)
    return backg

In [29]:
# Helper function.
def save_file(filename, image, cmap):
    """Save new image using filename as base."""
    print('image shape:', image.shape)
    print('cmap:', cmap)
    mx = np.max(image)
    image = (image * 255 / mx).astype(np.uint8)
    i = filename.find('.')
    plt.imsave(filename[:i] + 'stereogram.png', image, cmap=cmap)

In [30]:
# Helper function.
def show_image(m_rows, n_cols, cmap, image):
    """Show the image."""
    # Estimates a good figure size.
    fig_sz = int(max([10 * m_rows / n_cols, 10]))
    print('figure size:', fig_sz)
    print(f'cmap: {cmap}')
    fig = plt.figure(figsize=(fig_sz, fig_sz))
    plt.imshow(image, cmap=cmap)
    plt.axis("off")
    plt.show()

In [31]:
g3x2=(3,2)
g2x3=(2,3)
g2x2=(2,2)
def average_pixels(grid_original, weight=4): #"""((0 1 2) (3 4 5) (6 7 8))"""
    grid=grid_original.copy()
    match weight:
        case 0:
            grid*=np.array((4,2,2,1)).reshape(g2x2)/9
        case 1:
            grid*=np.array((2,2,2,1,1,1)).reshape(g2x3)/9
        case 2:
            grid*=np.array((2,4,1,2)).reshape(g2x2)/9
        case 3:
            grid*=np.array((2,1,2,1,2,1)).reshape(g3x2)/9
        case 4:
            grid/=9
        case 5:
            grid*=np.array((1,2,1,2,1,2)).reshape(g3x2)/9
        case 6:
            grid*=np.array((2,1,4,2)).reshape(g2x2)/9
        case 7:
            grid*=np.array((1,1,1,2,2,2)).reshape(g2x3)/9
        case 8:
            grid*=np.array((1,2,2,4)).reshape(g2x2)/9
        case _:
            # No case matches
            grid/=9
    return np.sum(grid)

def blur_pixels(grid):
    grid_copy=grid.copy()
    (x,y)=grid.shape
    for i in range(x):
        for j in range(y):
            arr = grid_copy[max(0,i-1):i+2,max(0,j-1):j+2]
            grid[i,j]=average_pixels(arr, 3*(min(1,i) + min(1,max(i-x+2,0))) 
                                     + (min(1,j) + min(1,max(j-y+2,0))))

In [32]:
# This box holds a function (a faster random-dot stereogram generator) using an inner function.
def stereogram(filename='saturn.png', stripe=None, max_shift=140,
               cmap=None, blurring=False, saving_file=False, showing=False):
    """Makes Autostereograms, also known as Magic-eye Illusions
    Note: Max depth and min depth are normalized"""
    # Image of overlay, with grayscale brightness being the depth.
    depth_map = image_read(filename)
    # Would like to scale the depth according to max possible depth
    uint8_max = 256 if np.issubdtype(depth_map.dtype, np.uint8) else None
    he, wi = depth_map.shape

    # 1/6 total width is for far offset, 1/8 for near offset
    # They scale with depth map resolution
    far_wi, near_wi = wi // 4, wi // 8
    min_shift = far_wi - near_wi
    if saving_file or showing:
        print(f'far width shift: {far_wi}, near width shift: {near_wi}, '
              f'diff: {min_shift}, upper bound shift: {max_shift}')
    
    # Max pixel shift is not more than max_shift argument
    shift_diff = np.min((min_shift, max_shift))
    map_min, map_max = np.min(depth_map), np.max(depth_map)
    map_diff = map_max - map_min
    print(f'depth map min: {map_min}, depth map max: {map_max}')
    # Why uint8? Because sometimes you only want subtle depth.
    if uint8_max is not None:
        shift_diff *= map_diff / uint8_max
    
    # offset_map is array of image shape with integer values in [0, shift_diff].
    offset_map = np.round(shift_diff * ((depth_map - map_min) / map_diff)).astype(int)
    print(f'new min: {np.min(offset_map)}, new max: {np.max(offset_map)}')
    
    # n_cols is for figure size only
    m_rows, n_cols = he, wi + shift_diff
    background = make_background(stripe, cmap, m_rows, far_wi)
    print(f'strip shape: {background.shape}')
    
    image = gen_rows(stripe, background, offset_map, m_rows, wi, cmap)
    if blurring:
        blur_pixels(image)
    
    if saving_file:
        save_file(filename, image, cmap)

    if showing:
        show_image(m_rows, n_cols, cmap, image)

In [33]:
"""
Primary Color maps:
viridis, plasma, winter, copper, coolwarm, 
ocean, gist_earth, terrain, gist_stern, gnuplot, gnuplot2, CMRmap, 
cubehelix, gist_rainbow, rainbow, nipy_spectral, gist_ncar, 
flag, pink, PiYG, PRGn, BrBG, PuOr, RdGy, RdBu, RdYlBu, RdYlGn, Spectral, bwr
"""

cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis',
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper',
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
            'twilight', 'twilight_shifted', 'hsv',
#             'Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2', 
#             'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b', 'tab20c', 'flag', 'prism', 
            'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
            'gist_ncar']

In [34]:
"""
CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=72063
"""
stripe = imageio.imread('RidingTigersPattern.png')
# fig = plt.figure(figsize=(10,10))
# plt.imshow(stripe)
# plt.show()

In [ ]:
# This box will produce a stereogram.
# Type the name of the file for the depth map.
# Also type the name of the file for the stripe pattern. If None, it will produce a colormap.

# depth_map = 'globe.jpg'
# depth_map = 'smiley1.png'
depth_map = 'saturn.png'
# stripe = 'RidingTigersPattern.png'
stripe = None

if stripe is not None:
    stripe = imageio.imread(stripe)

# To get the stereogram.
maxd = 45
coloring = 'YlOrRd'
# coloring = 'viridis'
blurring = True
saving = False
showing = True

stereogram(depth_map, stripe=stripe, max_shift=maxd, 
           cmap=coloring, blurring=blurring, saving_file=saving, showing=showing)

# for cmap in cmaps:
#     stereogram('map.png', max_shift=maxd, cmap=cmap, saving_file=saving, showing=showing)